Импортируем все необходимые библиотеки

In [109]:
pip install nltk natasha scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [110]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import pymorphy2

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd

In [113]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Выделение обозначения стоп-слов

In [116]:
stop_words = set(stopwords.words('russian'))

In [118]:
texts = [
    "Мама мыла раму и пела песню.",
    "Сегодня прекрасная погода, и мы пошли гулять в парк.",
    "Студенты изучают программирование на Python и создают интересные проекты."
]

Функция лемматизации текста

In [121]:
def lemm(text):
    
    segmenter = Segmenter()
    Vocab = MorphVocab()
    Embeddings = NewsEmbedding()
    Tagger = NewsMorphTagger(Embeddings)
    Stemmer = SnowballStemmer("russian")
    morph = pymorphy2.MorphAnalyzer()

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(Tagger)
    
    lems = []
    for token in doc.tokens:
        if token.lemma and token.lemma.isalnum():  
            lems.append(token.lemma)
        else:
            parsed = morph.parse(token.text)[0]
            if parsed.normal_form.isalnum():
                lems.append(parsed.normal_form)
    
    return lems

In [123]:
lemms = [lemm(text) for text in texts]

Функция токенизации текста(без стоп-слов)

In [125]:
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

In [126]:
tokens = [tokenize_text(text) for text in texts]

Функция поиска стоп-слов(по аналогии с токенизацией, но отдельная функция):

In [128]:
def stop_words_search(text):
    stopwords = word_tokenize(text.lower())
    return [word for word in stopwords if word in stop_words]

In [129]:
stopwords = [stop_words_search(text) for text in texts]

Вывод результатов:

In [131]:
print("Результаты токенизации(сразу с удалением стоп-слов):")
print(tokens)

print("Результаты поиска стоп-слов:")
print(stopwords)

print("Результаты ламматизации:")
print(lemms)

Результаты токенизации(сразу с удалением стоп-слов):
[['мама', 'мыла', 'раму', 'пела', 'песню'], ['сегодня', 'прекрасная', 'погода', 'пошли', 'гулять', 'парк'], ['студенты', 'изучают', 'программирование', 'python', 'создают', 'интересные', 'проекты']]
Результаты поиска стоп-слов:
[['и'], ['и', 'мы', 'в'], ['на', 'и']]
Результаты ламматизации:
[['мама', 'мыло', 'рама', 'и', 'петь', 'песня'], ['сегодня', 'прекрасный', 'погода', 'и', 'мы', 'послать', 'гулять', 'в', 'парк'], ['студент', 'изучать', 'программирование', 'на', 'python', 'и', 'создавать', 'интересный', 'проект']]


In [137]:
def bag_of_words(texts):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    result = X.toarray()
    
    return feature_names, result

In [140]:
bow_features, bow_matrix = bag_of_words(texts)

#красивый вывод на табличке
bow_df = pd.DataFrame(bow_matrix, columns=bow_features)
bow_df.index = [f"Предложение {i+1}" for i in range(len(texts))]

In [142]:
def tf_idf(texts):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    result = X.toarray()
    
    return feature_names, result

In [144]:
tfidf_features, tfidf_matrix = tf_idf(texts)

#красивый вывод на табличке
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf_features)
tfidf_df.index = [f"Предложение {i+1}" for i in range(len(texts))]

In [146]:
print("Bag of Words:")
display(bow_df)

Bag of Words:


,python,гулять,изучают,интересные,мама,мы,мыла,на,парк,пела,песню,погода,пошли,прекрасная,программирование,проекты,раму,сегодня,создают,студенты
Предложение 1,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0
Предложение 2,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,0
Предложение 3,1,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1


In [148]:
print("\nTF-IDF:")
display(tfidf_df)


TF-IDF:


,python,гулять,изучают,интересные,мама,мы,мыла,на,парк,пела,песню,погода,пошли,прекрасная,программирование,проекты,раму,сегодня,создают,студенты
Предложение 1,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.447214,0.000000,0.000000,0.447214,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000
Предложение 2,0.000000,0.377964,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000,0.377964,0.000000,0.000000,0.377964,0.377964,0.377964,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000
Предложение 3,0.353553,0.000000,0.353553,0.353553,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.353553,0.000000,0.000000,0.353553,0.353553
